In [3]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
from os import urandom

In [5]:
def bytes_to_bitstring(data: bytes) -> str:
    return ''.join(f'{b:08b}' for b in data)

def bitstring_to_bytes(bitstr: str) -> bytes:
    return int(bitstr, 2).to_bytes(len(bitstr) // 8, byteorder='big')

def flip_bits(data: bytes) -> bytes:
    return bytes(~b & 0xFF for b in data)

def des_encrypt_ecb(key: bytes, plaintext: bytes) -> bytes:
    cipher = DES.new(key, DES.MODE_ECB)
    return cipher.encrypt(plaintext)

In [6]:
# Permuted Choice 1 (PC-1): 64-bit key to 56-bit key
PC1 = [
    57, 49, 41, 33, 25, 17,  9,
     1, 58, 50, 42, 34, 26, 18,
    10,  2, 59, 51, 43, 35, 27,
    19, 11,  3, 60, 52, 44, 36,
    63, 55, 47, 39, 31, 23, 15,
     7, 62, 54, 46, 38, 30, 22,
    14,  6, 61, 53, 45, 37, 29,
    21, 13,  5, 28, 20, 12,  4
]

# Permuted Choice 2 (PC-2): 56-bit to 48-bit key
PC2 = [
    14, 17, 11, 24,  1,  5,
     3, 28, 15,  6, 21, 10,
    23, 19, 12,  4, 26,  8,
    16,  7, 27, 20, 13,  2,
    41, 52, 31, 37, 47, 55,
    30, 40, 51, 45, 33, 48,
    44, 49, 39, 56, 34, 53,
    46, 42, 50, 36, 29, 32
]

# Number of left shifts per round
SHIFT_SCHEDULE = [
    1, 1, 2, 2, 2, 2, 2, 2,
    1, 2, 2, 2, 2, 2, 2, 1
]

def left_shift(bits, n):
    return bits[n:] + bits[:n]

def permute(bits, table):
    return ''.join(bits[i - 1] for i in table)

def generate_round_keys(key_64bit):
    # Step 1: apply PC-1 to get 56-bit key
    key_64bit = ''.join(f'{int(b):08b}' for b in key_64bit) if isinstance(key_64bit, bytes) else key_64bit
    key_56bit = permute(key_64bit, PC1)

    # Step 2: split into two 28-bit halves
    C = key_56bit[:28]
    D = key_56bit[28:]

    round_keys = []

    for shift in SHIFT_SCHEDULE:
        # Step 3: left shift both halves
        C = left_shift(C, shift)
        D = left_shift(D, shift)

        # Step 4: combine and apply PC-2
        combined = C + D
        round_key = permute(combined, PC2)
        round_keys.append(round_key)

    return round_keys

# Example usage:
# A 64-bit key as a binary string (normally you'd convert from a hex or byte string)
dat=urandom(8)
sample_key =  bytes_to_bitstring(dat)   # 64 bits
round_keys = generate_round_keys(sample_key)

# Print round keys
for i, rk in enumerate(round_keys, 1):
    print(f'Round {i}: {rk}')


Round 1: 001100001001101001010000100111010010100001001001
Round 2: 101000000101100000010000011000010000011011100101
Round 3: 000001000010001101010100110110101000100010001011
Round 4: 000001100101010000010001000001100101011100011101
Round 5: 010011110000000101100000000110110011000111100000
Round 6: 100010101100000010001001111000001100100100100001
Round 7: 000110010000001100001010010000100010111000011110
Round 8: 001000000001100010001001111111010001000110011000
Round 9: 000110000011000010000000100011111010011000100100
Round 10: 000100000000110001011100011110000100111111000000
Round 11: 010001000110000000000100000110001100000000011111
Round 12: 000000101000110100100100110001110111010010000000
Round 13: 110010000010000000100011101010000010001101101001
Round 14: 101000011000011000101000101100101101101000000110
Round 15: 110000000001001010000010010101000000011110110010
Round 16: 001100000000001001100010100001001111000110010100
